In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
%config InlineBackend. figure_format = 'retina'


In [15]:
all= pd.read_csv('../AllWindowData_relative.csv')
data=pd.DataFrame({'id':all["userId"],"case":all['caseNum'], 'Window Type':all['type'],'Environment': all[ "environment"],'Distance':all["distance"],'Window Anchor':all['windowMode'],'Transparency':all['transparency'],'Size':all['visual angle'],'Meeting Format':all['Meeting type'], 'theta':all['theta'], 'phi':all['phi'], 'Offset':all['offset'], 'x': all['x'], 'y': all['y'], 'z': all['z'], 'scale': all['scale'], 'verlen': all['verlen'], 'horlen': all['horlen']})
data['Window Type'] = data['Window Type'].replace({0: 'Shared Screen', 1: 'Speaker\'s Video'})
data['Window Anchor'] = data['Window Anchor'].replace({0: 'Path Anchor', 1: 'Head Anchor'})
data['Meeting Format'] = data['Meeting Format'].replace({1: 'Listening', 2: 'Discussion'})
data['Environment'] = data['Environment'].replace({'low': 'Low-Traffic', 'high': 'High-Traffic'})
data['Transparency']=1-data['Transparency']

In [16]:
# 分上下組
downID = [0,   1,   2,   3,   4,   5,   6,   7,   9,  10,  11,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  48,
        49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
        62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  80,  81,  82,
        83,  84,  85,  86,  87,  97,  98,  99, 103, 104, 105, 106, 107,
       109, 111, 112, 113, 114, 115, 116, 117, 118, 119]
downID = [s*2 for s in downID] + [s*2+1 for s in downID]
data_down = data.loc[downID].reset_index()
# print(data_down)
upID = [8,  12,  40,  41,  42,  43,  44,  45,  46,  47,  72,  73,  74,
        75,  76,  77,  78,  79,  88,  89,  90,  91,  92,  93,  94,  95,
        96, 100, 101, 102, 108, 110, 120, 121, 122, 123, 124, 125, 126,
       127]
upID = [s*2 for s in upID] + [s*2+1 for s in upID]
data_up = data.loc[upID].reset_index()

In [17]:
def get_3d_data(data, para, updown):
    result = {
        "playerId": para + ' ' + updown,
        "participant": [],
        "presentation": [],
        "time": [],
        "velocity": []
    }
    grouped = data.groupby([para, 'Window Type'])['Transparency', 'scale', 'x', 'y', 'z', 'verlen', 'horlen'].mean().reset_index()
    print(grouped)
    for i in range(4):
        element = {
            "position": [grouped['x'][i], grouped['y'][i], grouped['z'][i]],
            "rotation": [0, 0, 0],
            "scale": [grouped['scale'][i], grouped['scale'][i], 1.0],
            "time": grouped[para][i],
            "transparency": grouped['Transparency'][i],
            "windowType": True if grouped['Window Type'][i] == "Shared Screen" else False,
        }
        if element['windowType']:
            element["rotation"] = element["position"]
            result['presentation'].append(element)
        else:
            element['position'] = [0.30293, -0.018717, -0.051715]
            result['participant'].append(element)
    # print(result)
    with open('./3d_result/' + para + '_' + updown + '.json', 'w') as json_file:
        json.dump(result, json_file, indent=4)

def get_3d_data_ud(data, updown):
    result = {
        "playerId": updown,
        "participant": [],
        "presentation": [],
        "time": [],
        "velocity": []
    }
    grouped = data.groupby(['Window Type'])['Transparency', 'scale', 'x', 'y', 'z'].mean().reset_index()
    print(grouped)
    for i in range(2):
        element = {
            "position": [grouped['x'][i], grouped['y'][i], grouped['z'][i]],
            "rotation": [0, 0, 0],
            "scale": [grouped['scale'][i], grouped['scale'][i], 1.0],
            "time": '',
            "transparency": grouped['Transparency'][i],
            "windowType": True if grouped['Window Type'][i] == "Shared Screen" else False,
        }
        if element['windowType']:
            element["rotation"] = element["position"]
            result['presentation'].append(element)
        else:
            element['position'] = [0.30293, -0.018717, -0.051715]
            result['participant'].append(element)
    # print(result)
    with open('./3d_result/' + updown + '.json', 'w') as json_file:
        json.dump(result, json_file, indent=4)



In [18]:
get_3d_data_ud(data_down, 'down')
get_3d_data_ud(data_up, 'up')
get_3d_data(data_down, 'Environment', 'down')
get_3d_data(data_up, 'Environment', 'up')
get_3d_data(data_down, 'Meeting Format', 'down')
get_3d_data(data_up, 'Meeting Format', 'up')
get_3d_data(data_down, 'Window Anchor', 'down')
get_3d_data(data_up, 'Window Anchor', 'up')

       Window Type  Transparency     scale         x         y         z
0    Shared Screen      0.813586  1.493344 -0.109337 -0.413565  0.781609
1  Speaker's Video      0.811396  0.570481  0.179330  0.058401 -0.081915
       Window Type  Transparency     scale         x         y         z
0    Shared Screen      0.841239  1.371184  0.032510  0.113466  0.705342
1  Speaker's Video      0.831581  0.529222 -0.040476 -0.084845 -0.096213
    Environment      Window Type  Transparency     scale         x         y  \
0  High-Traffic    Shared Screen      0.802339  1.454440 -0.114519 -0.413527   
1  High-Traffic  Speaker's Video      0.735612  0.579356  0.166946  0.069356   
2   Low-Traffic    Shared Screen      0.823855  1.528865 -0.104605 -0.413601   
3   Low-Traffic  Speaker's Video      0.880589  0.562378  0.190636  0.048398   

          z     verlen     horlen  
0  0.790761  18.576856  33.095442  
1 -0.115043  11.289487  11.167116  
2  0.773252  20.337462  35.973211  
3 -0.051669  11.0

C:\Users\chiao6673\AppData\Local\Temp\ipykernel_20996\3645794798.py:38: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped = data.groupby(['Window Type'])['Transparency', 'scale', 'x', 'y', 'z'].mean().reset_index()
C:\Users\chiao6673\AppData\Local\Temp\ipykernel_20996\3645794798.py:38: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped = data.groupby(['Window Type'])['Transparency', 'scale', 'x', 'y', 'z'].mean().reset_index()
C:\Users\chiao6673\AppData\Local\Temp\ipykernel_20996\3645794798.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped = data.groupby([para, 'Window Type'])['Transparency', 'scale', 'x', 'y', 'z', 'verlen', 'horlen'].mean().reset_index()
C:\Users\chiao6673\AppData\Local\Temp\ipykernel_20996\3645794798.py:9: Futu

In [11]:
# p = data[(data['Window Type']=='Shared Screen')]
# print(p)
for index, row in data.iterrows():
    if row['Window Type'] == "Speaker's Video":
        pass
    elif index in upID:
        print(row['id'], row["case"], row["Window Anchor"], "up")
    elif index in downID:
        print(row['id'], row["case"], row["Window Anchor"], "down")


1 1 Path Anchor down
1 1 Head Anchor down
1 2 Path Anchor down
1 2 Head Anchor down
1 3 Path Anchor down
1 3 Head Anchor down
1 4 Path Anchor down
1 4 Head Anchor down
2 1 Path Anchor up
2 1 Head Anchor down
2 2 Path Anchor down
2 2 Head Anchor down
2 3 Path Anchor up
2 3 Head Anchor down
2 4 Path Anchor down
2 4 Head Anchor down
3 1 Path Anchor down
3 1 Head Anchor down
3 2 Path Anchor down
3 2 Head Anchor down
3 3 Path Anchor down
3 3 Head Anchor down
3 4 Path Anchor down
3 4 Head Anchor down
4 1 Path Anchor down
4 1 Head Anchor down
4 2 Path Anchor down
4 2 Head Anchor down
4 3 Path Anchor down
4 3 Head Anchor down
4 4 Path Anchor down
4 4 Head Anchor down
5 1 Path Anchor down
5 1 Head Anchor down
5 2 Path Anchor down
5 2 Head Anchor down
5 3 Path Anchor down
5 3 Head Anchor down
5 4 Path Anchor down
5 4 Head Anchor down
6 1 Path Anchor up
6 1 Head Anchor up
6 2 Path Anchor up
6 2 Head Anchor up
6 3 Path Anchor up
6 3 Head Anchor up
6 4 Path Anchor up
6 4 Head Anchor up
7 1 Path Anc

In [18]:
selected_id = [0,   2,   4,   6,  16,  17,  18,  19,  20,  21,  24,  25,  28,
        29,  32,  33,  34,  38,  39,  48,  49,  51,  52,  53,  54,  55,
        56,  58,  60,  96,  97,  98,  99, 100, 101, 102, 103, 105, 106,
       107, 108, 109, 110, 111, 112, 114, 115, 120, 121, 122, 123]
selected_id = [s*2 for s in selected_id]
data_relative = data.loc[selected_id].reset_index()
print(data_relative)
grouped = data_relative.groupby(['Window Type'])['Transparency', 'scale', 'x', 'y', 'z'].mean().reset_index()#.agg(mean='mean')#.values.tolist()
print(grouped)

    index  id  case      Window Type   Environment  Distance Window Anchor  \
0       0   1     1  Speaker's Video   Low-Traffic  0.267940   Path Anchor   
1       4   1     2  Speaker's Video   Low-Traffic  0.267940   Path Anchor   
2       8   1     3  Speaker's Video  High-Traffic  0.267940   Path Anchor   
3      12   1     4  Speaker's Video  High-Traffic  0.267940   Path Anchor   
4      32   3     1  Speaker's Video   Low-Traffic  0.350904   Path Anchor   
5      34   3     1  Speaker's Video   Low-Traffic  0.324316   Head Anchor   
6      36   3     2  Speaker's Video   Low-Traffic  0.368168   Path Anchor   
7      38   3     2  Speaker's Video   Low-Traffic  0.368168   Head Anchor   
8      40   3     3  Speaker's Video  High-Traffic  0.294471   Path Anchor   
9      42   3     3  Speaker's Video  High-Traffic  0.324167   Head Anchor   
10     48   4     1  Speaker's Video   Low-Traffic  0.248049   Path Anchor   
11     50   4     1  Speaker's Video   Low-Traffic  0.248049   H

C:\Users\chiao6673\AppData\Local\Temp\ipykernel_15252\1094397484.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped = data_relative.groupby(['Window Type'])['Transparency', 'scale', 'x', 'y', 'z'].mean().reset_index()#.agg(mean='mean')#.values.tolist()
